In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [85]:
# ocpf filings under filer 10176 Galvin, William F.
ocpf_10176 = pd.read_excel('ocpf-output-10176.xlsx')
ocpf_10176.head()

,Date,Contributor,Address,City,State,Zip,Occupation,Employer,Principal Officer,Amount,CPF ID,Recipient,Tender Type ID,Tender Type Description,Record Type ID,Record Type Description,Source Description
0,8/12/2002,"Cusack, Francis W.",351 Nahant Road,Nahant,MA,01908,NaN,NaN,NaN,100.0,10176,"Galvin, William F.",1,Check,201,Individual,8/12/02 Deposit Report
1,8/12/2002,Sheet Metal Workers Local #17,1157 Adams Street,Dorchester,MA,02124,NaN,NaN,NaN,200.0,10176,"Galvin, William F.",1,Check,203,Union/Association,8/12/02 Deposit Report
2,8/19/2002,Mass. Laborers Dist. Council,7 Laborers Way,Hopkinton,MA,01748,NaN,NaN,NaN,500.0,10176,"Galvin, William F.",1,Check,203,Union/Association,8/19/02 Deposit Report
3,8/29/2002,"Dennis, Sr., Richard J.",6 Beacon Street,Boston,MA,02108,Real Estate Broker,NaN,NaN,250.0,10176,"Galvin, William F.",1,Check,201,Individual,8/29/02 Deposit Report
4,8/29/2002,"Healey, David M.",107 Academy Hill Road,Brighton,MA,02135,NaN,NaN,NaN,100.0,10176,"Galvin, William F.",1,Check,201,Individual,8/29/02 Deposit Report


In [86]:
# ocpf filings under filer 20176 Galvin, William F.
ocpf_20176 = pd.read_excel('ocpf-output-20176.xlsx')
ocpf_20176.head()

,Date,Contributor,Address,City,State,Zip,Occupation,Employer,Principal Officer,Amount,CPF ID,Recipient,Tender Type ID,Tender Type Description,Record Type ID,Record Type Description,Source Description
0,1/28/2002,"Dicorato, Elaine",2 Pebble Place,Stoneham,MA,2180.0,NaN,NaN,NaN,100,20176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
1,1/28/2002,"O'Toole, Feargal","59 Oxford St., Apt. #2",Somerville,MA,2143.0,NaN,NaN,NaN,100,20176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
2,4/17/2002,Aggregated Unitemized Receipts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,20176,"Galvin, William F.",0,NaN,220,Aggregated Unitemized Receipts,4/17/02 Deposit Report
3,4/17/2002,"Casey, Christopher",2 Homestead Road,Marblehead,MA,1945.0,NaN,NaN,NaN,100,20176,"Galvin, William F.",1,Check,201,Individual,4/17/02 Deposit Report
4,4/17/2002,"Cutter, Simon","Presidential Park, 314 Main Street",Wilmington,MA,1887.0,NaN,NaN,NaN,100,20176,"Galvin, William F.",1,Check,201,Individual,4/17/02 Deposit Report


In [87]:
# ocpf filings under filer 30176 Galvin, William F.
ocpf_30176 = pd.read_excel('ocpf-output-30176.xlsx')
ocpf_30176.head()

,Date,Contributor,Address,City,State,Zip,Occupation,Employer,Principal Officer,Amount,CPF ID,Recipient,Tender Type ID,Tender Type Description,Record Type ID,Record Type Description,Source Description
0,1/28/2002,"Cheyne, R. Scott",73 High Ridge,Boxford,MA,1921,Exec. Hill Holiday,NaN,NaN,500,30176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
1,1/28/2002,"Farrington, Dennis",200 Clarendon Street,Boston,MA,2116,Exec. Hill. Holiday,NaN,NaN,500,30176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
2,1/28/2002,"Kay, Deborah",98 Musquash Road,Hudson,NH,3051,at home,NaN,NaN,500,30176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
3,1/28/2002,"Kay, Richard",98 Musquash Road,Hudson,NH,3051,self-employed,NaN,NaN,500,30176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report
4,1/28/2002,"Koravos, Dean",99 Dunstable Road,Westford,MA,1886,sales,NaN,NaN,500,30176,"Galvin, William F.",1,Check,201,Individual,1/28/02 Deposit Report


In [88]:
total_ocpf = pd.concat([ocpf_10176, ocpf_20176,ocpf_30176], ignore_index=True)
total_ocpf.Contributor = total_ocpf.Contributor.astype(str).apply(lambda x: x.replace(', Sr.', ' Sr.'))
total_ocpf.Contributor = total_ocpf.Contributor.astype(str).apply(lambda x: x.replace(', Jr.', ' Jr.'))

In [89]:
total_ocpf.Contributor=total_ocpf.Contributor.astype(str).apply(lambda x: x.replace('\n',""))
total_ocpf.groupby(by="Contributor").sum("Amount").sort_values(by='Amount',ascending=False)

,Amount,CPF ID,Tender Type ID,Record Type ID
Contributor,,,,
Bank of America,430130.88,10176,4,201
Skdknickerbocker LLC,58981.43,10176,1,204
"Haivanis, George",13525.00,213696,21,4221
Aggregated Unitemized Receipts,13123.15,691792,0,14740
"Rachins, Edward A.",13000.00,193344,19,3819
...,...,...,...,...
"Hall, Alice N.",25.00,10176,1,201
"Nolet, Linda M.",25.00,10176,1,201
"Elms, Jessica M",20.00,10176,1,201


In [90]:
total_ocpf[total_ocpf['Principal Officer'].notna() ]

,Date,Contributor,Address,City,State,Zip,Occupation,Employer,Principal Officer,Amount,CPF ID,Recipient,Tender Type ID,Tender Type Description,Record Type ID,Record Type Description,Source Description
1141,9/17/2004,Laborers Int'l Union of No. America Const. & Gen.,P.O. Box 128,Taunton,MA,02780,NaN,NaN,"Bus. Mg., Treas.",125.0,10176,"Galvin, William F.",1,Check,203,Union/Association,9/17/04 Deposit Report
1700,12/17/2004,Nat'l Assoc. of Government Employees,129 Enponset Avenue,Dorchester,MA,02122,NaN,NaN,Sec./Treasurer,500.0,10176,"Galvin, William F.",1,Check,203,Union/Association,12/17/04 Deposit Report
1842,2/28/2005,ABCC Investigators Assoc.,90 Homer Street,East Boston,MA,02128,NaN,NaN,Treasurer,250.0,10176,"Galvin, William F.",1,Check,203,Union/Association,2/28/05 Deposit Report
2408,11/4/2005,Greater Southeastern MA Labor Council,867 State Road,North Dartmouth,MA,02747,NaN,NaN,Treasurer,100.0,10176,"Galvin, William F.",1,Check,203,Union/Association,11/4/05 Deposit Report
2847,4/28/2006,Millwrights Local 1121,90 Braintree St.,Allston,MA,02134,NaN,NaN,Treasuer,500.0,10176,"Galvin, William F.",1,Check,203,Union/Association,4/28/06 Deposit Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9402,3/29/2018,NAGE R1-207,159 Burgin Parkway,Quincy,MA,02169,NaN,NaN,"Theresa McGoldrick, Nat'l Exec. V.P.",1000.0,10176,"Galvin, William F.",1,Check,203,Union/Association,3/29/18 Deposit Report
9403,3/29/2018,NAGE R1-292,159 Burgin Parkway,Quincy,MA,02169,NaN,NaN,Donna M. Jackson,1000.0,10176,"Galvin, William F.",1,Check,203,Union/Association,3/29/18 Deposit Report
10943,11/30/2020,Boston Firefighters Local 718 (10/15),55 Hallet Street,Dorchester,MA,02124,NaN,NaN,John R. Soares,500.0,10176,"Galvin, William F.",1,Check,203,Union/Association,11/30/20 Deposit Report
11005,2/16/2021,Teamsters Local Union No. 122,765 East Third Street,Boston,MA,02127,NaN,NaN,John Murphy,250.0,10176,"Galvin, William F.",1,Check,203,Union/Association,2/16/21 Deposit Report


In [91]:
tax_credits = pd.read_excel('round_53_17_latest_round.xlsx')
tax_credits = tax_credits.rename(columns={'Name of Property': 'property_name',
                            'Prior Rounds Applications': 'prior_rounds_applications',
                            'Proponent Name': 'proponent_name',
                            'Remaining Credit to Award': 'remaining_credit_to_award',
                            'Total Awards': 'total_awards',
                            'Part 2 Qualified Rehab. $':'part_2_qualified_rehab_$',
                            'Part 2 Total Project Cost': 'part_2_total_project_cost'})

# formatting name to match names in OCPF records 
tax_credits.proponent_name = tax_credits.proponent_name.apply(lambda x: x.split(' ')[-1] + ', '+ ' '.join(x.split(' ')[:-1]))
tax_credits

,Unnamed: 0,City/Town,property_name,Use**,prior_rounds_applications,proponent_name,remaining_credit_to_award,total_awards,20% QRE,Qualified Rehab. $,Total Project Cost,HRC #,Entity,part_2_qualified_rehab_$,part_2_total_project_cost,Count_prior_rounds_applied
0,2431,Lawrence,Wood Worsted Mill Building C (formerly Office ...,R,"13 4, 15, 16,17, 18, 19, 20,21, 22, 23, 24,...","Lupoli, Salvatore",574607.60,3000000,3574607.60,17873038.0,22481348.0,117.0,200 Riverwalk LLC,NaN,NaN,21.0
1,805,Lowell,Hamilton Canal Lofts (Phase 2 of 2),"R, C","6,7,8,9,10,11,12,13,14,15, 16,17, 18, 19, 20,2...","McDonnell, Sean",585826.40,12800000,7485826.40,37429132.0,45006500.0,54.0,"HCL Acquisition LLC, subsidiary of Architectur...",NaN,NaN,21.0
2,1234,Waltham,Waltham Watch Company Phased (Phases 10-11 of 12),"R, C","11, 12, 13, 14,15, 16, 17, 18,19, 20, 21, 22,2...","McGill, Ian",4259000.00,4700000,22729452.80,113647264.0,132824000.0,96.0,"Watch City Ventures LLC, c/o Berkeley Investme...",NaN,NaN,20.0
3,1350,Boston / Charlestown,"Terminal Storage Company, 267 Medford Street",R,"12,13, 15, 16,17, 18, 19, 21,22, 23, 24, 25,26...","Rauseo, Michael",1740000.00,6700000,5640000.00,28200000.0,37200000.0,103.0,Suffolk Medford LLC,NaN,NaN,19.0
4,1926,Fitchburg,"Fitchburg Yarn Mill, 1428 Main Street",R,"21, 22, 23, 24,25, 26, 27, 28,29, 30, 31, 32,3...","Curtis, Lawrence",755994.80,4900000,5655994.80,28279974.0,37440936.0,275.0,Fitchburg Yarn Limited Partnership,NaN,NaN,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,1955,Salem,"Pickering Dodge House, 35 Chestnut Street",R,NaN,"Gillespie, Philip",251870.00,0,251870.00,1259350.0,2094330.0,763.0,NaN,NaN,NaN,NaN
464,1967,Springfield,"New Court Terrace: FrederickB. Taylor Block, 7...",R,38,"Gagliardi, Peter",166766.00,0,166766.00,833830.0,1104750.0,742.0,"HAP, Inc.",NaN,NaN,NaN
465,1968,Springfield,"New Court Terrace: William Birnie Block, 68-70...",R,38,"Gagliardi, Peter",157086.40,0,157086.40,785432.0,1177776.0,743.0,"HAP, Inc.",NaN,NaN,NaN
466,2193,Haverhill,"St. Gregory School, 108 Harrison Street",R,40,"Lipchitz, William F.",1013757.40,0,1013757.40,5068787.0,8638007.0,788.0,Common Ground Development Corporation,NaN,NaN,NaN


In [92]:
# total_ocpf.to_excel('total_ocpf.xlsx')

In [93]:
# test = ocpf_10176.Employer.unique()
# list(test)